In [ ]:
import os
import subprocess
import time
import shutil
import pandas as pd
import multiprocessing

# Configuration
n_threads = multiprocessing.cpu_count()
os.environ['OMP_NUM_THREADS'] = str(n_threads)
print(f"Using {n_threads} threads")

# Parameters
n_steps = 20000
r_restarts = 16
loop_iterations = 3 # Run the loop 3 times

def run_tree_packer(iteration):
    print(f"\n--- Iteration {iteration}: Running Tree Packer ---")
    cmd = ["./tree_packer.exe", "-n", str(n_steps), "-r", str(r_restarts)]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    final_score = None
    for line in process.stdout:
        print(line, end='')
        if "Final:" in line:
            try:
                final_score = float(line.split()[1])
            except:
                pass
                
    process.wait()
    
    if process.returncode != 0:
        print("Tree Packer failed.")
        return False
    
    # Check if output file exists (submission_v21.csv based on previous knowledge, or maybe it overwrites submission.csv?)
    # The code seems to output to submission_v21.csv usually, but let's check.
    # Actually, looking at the code dump in previous turn, I didn't see the output filename.
    # But the python wrapper in the reference notebook expected `submission_{version}.csv`.
    # Let's assume it outputs `submission_v21.csv` or similar.
    # I'll check for any new CSVs.
    
    output_file = "submission_v21.csv" # Default in reference
    if os.path.exists(output_file):
        print(f"Found {output_file}. Updating submission.csv")
        shutil.copy(output_file, "submission.csv")
        return True
    elif os.path.exists("submission.csv"):
        # Maybe it updated submission.csv directly?
        print("No submission_v21.csv found. Checking if submission.csv was modified...")
        return True
    else:
        print("No output file found.")
        return False

def run_bp(iteration):
    print(f"\n--- Iteration {iteration}: Running Backward Propagation ---")
    cmd = ["./bp.exe", "submission.csv", "submission_bp.csv"]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    for line in process.stdout:
        print(line, end='')
        
    process.wait()
    
    if process.returncode == 0 and os.path.exists("submission_bp.csv"):
        print("BP finished. Updating submission.csv")
        shutil.copy("submission_bp.csv", "submission.csv")
        return True
    else:
        print("BP failed.")
        return False

# Main Loop
for i in range(loop_iterations):
    print(f"Starting Loop {i+1}/{loop_iterations}")
    
    # Step 1: Tree Packer
    if not run_tree_packer(i+1):
        break
        
    # Step 2: BP
    if not run_bp(i+1):
        break
        
print("Optimization Loop Completed.")